In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import t
import matplotlib.pyplot as plt

In [2]:
np.random.seed(11)
ticker = "TRP.RO"
start_date = "2021-01-01"
confidence = 0.99
num_simulations = 50_000
horizon_days = 1

In [3]:
try:
    stock = yf.Ticker(ticker)
    df = stock.history(period="max")
except Exception as e:
    print("Error downloading", e)
    df = None
if df is None or df.empty:
    raise ValueError(f"No data returned for {ticker}")
if "Adj Close" in df.columns:
    df = df[["Adj Close"]].rename(columns={"Adj Close": "stock"})
else:
    df = df[["Close"]].rename(columns={"Close": "stock"})
#Use adjusted prices for VaR
#log returns
prices_log = np.log(1+df["stock"].pct_change())
prices_log.replace([np.inf, -np.inf], np.nan, inplace=True)
prices_log.dropna(inplace=True)
prices_log = np.asarray(prices_log)

In [4]:
prices = df["stock"].dropna()
returns = prices.pct_change().dropna()
current_price = float(prices.iloc[-1])
r = prices_log

In [5]:
print(df)
print(r)

                              stock
Date                               
2008-07-02 00:00:00+03:00  0.067571
2008-07-03 00:00:00+03:00  0.065241
2008-07-04 00:00:00+03:00  0.065241
2008-07-07 00:00:00+03:00  0.065241
2008-07-08 00:00:00+03:00  0.063688
...                             ...
2026-01-22 00:00:00+02:00  0.471000
2026-01-23 00:00:00+02:00  0.468000
2026-01-26 00:00:00+02:00  0.470500
2026-01-27 00:00:00+02:00  0.492500
2026-01-28 00:00:00+02:00  0.527000

[4520 rows x 1 columns]
[-0.03509668  0.          0.         ...  0.00532766  0.04569853
  0.06770609]


In [6]:
sampled = np.random.choice(r, size=(num_simulations, horizon_days), replace=True)
cumulative_return = np.prod(1 + sampled, axis=1) - 1
simulated_prices = current_price * (1 + cumulative_return)
pnl = simulated_prices - current_price
hb_var = -np.percentile(pnl, (1 - confidence) * 100)
#Student-t Monte Carlo VaR

In [7]:
dff, loc, scale = t.fit(r)
simulated_t = t.rvs(df=dff,loc=0,scale=scale,size=(num_simulations, horizon_days))
cumulative_return_t = np.prod(1 + simulated_t, axis=1) - 1
simulated_prices_t = current_price * (1 + cumulative_return_t)
pnl_t = simulated_prices_t - current_price
t_var = -np.percentile(pnl_t, (1 - confidence) * 100)

In [8]:
print(f"Last price: {current_price:.4f}")
print(f"Historical Bootstrap VaR (95%): {hb_var:.4f} RON")
print(f"On 99% of days, you should expect not to lose more than {(hb_var/current_price)*100:.4f}% in a day")
print(f"Student-t Monte Carlo VaR (95%): {t_var:.4f} RON")
print(f"On 99% of days, you should expect not to lose more than {(t_var/current_price)*100:.4f}% in a day")

Last price: 0.5270
Historical Bootstrap VaR (95%): 0.0498 RON
On 99% of days, you should expect not to lose more than 9.4487% in a day
Student-t Monte Carlo VaR (95%): 0.1147 RON
On 99% of days, you should expect not to lose more than 21.7702% in a day
